In [7]:
# pip install pillow numpy opencv-python
import numpy as np
from PIL import Image
import cv2

# =========================
# CONFIG
# =========================
INPUT   = "subject.png"                  # transparent PNG (cutout)
OUTPUT  = "subject_pointcloud_lidar.png" # transparent PNG

target_width   = 1200      # resize for density/speed
grid_step      = 1         # 1 = every pixel; 2/3 to thin base grid
depth_slices   = 48        # number of "planes" (more slices => smoother 3D)
depth_scale    = 0.55      # thickness of the cloud (0.2..0.7 good)
invert_depth   = True      # flip near/far if needed
gamma_depth    = 0.8       # <1 emphasises near; >1 emphasises far
smooth_sigma   = 2.0       # Gaussian blur for smooth depth (1.5..3.0 good)
unsharp_amount = 0.0       # 0..1 subtle pop in midtones (try 0.2)

# sampling
dup_particles      = 3     # duplicate points per sample for volume
slice_jitter_px    = 0.7   # random jitter (in image pixels) per duplicate
prob_from_brightness = 0.0 # 0..1: bias sampling toward brighter pixels

# projection (3D view)
rot_x_deg  = -6
rot_y_deg  = 10
focal_len  = 1100

# particle look
near_rgb   = np.array([255, 255, 255], dtype=np.uint8)  # white (near)
far_rgb    = np.array([ 80, 130, 255], dtype=np.uint8)  # blue  (far)
min_size   = 1     # px (far)
max_size   = 2     # px (near)
min_alpha  = 160   # far
max_alpha  = 255   # near

# =========================
# LOAD & PREP
# =========================
rgba = Image.open(INPUT).convert("RGBA")
w0, h0 = rgba.size
scale = target_width / w0
new_size = (target_width, int(round(h0 * scale)))
rgba = rgba.resize(new_size, Image.LANCZOS)
arr = np.array(rgba)
rgb = arr[..., :3].astype(np.float32) / 255.0  # 0..1
A   = arr[..., 3]                              # 0..255

H, W = A.shape

# =========================
# DEPTH FROM SMOOTHED LUMINANCE
# =========================
# Standard luminance (sRGB): Y = 0.2126 R + 0.7152 G + 0.0722 B
Y = (0.2126 * rgb[...,0] + 0.7152 * rgb[...,1] + 0.0722 * rgb[...,2]).astype(np.float32)

# Only consider subject area for stats
mask = (A > 0)
if mask.sum() == 0:
    raise ValueError("No alpha > 0 pixels found. Provide a cutout PNG.")

# Smooth luminance to get soft depth layers
Y_blur = cv2.GaussianBlur(Y, (0,0), smooth_sigma)

# Optional gentle unsharp mask to restore mid detail without noise
if unsharp_amount > 0:
    sharp = np.clip(Y + unsharp_amount * (Y - Y_blur), 0, 1)
else:
    sharp = Y_blur

# Normalize to [0, 1] over the subject only
vals = sharp[mask]
vmin, vmax = float(np.percentile(vals, 1)), float(np.percentile(vals, 99))
if vmax <= vmin:
    vmax = vmin + 1e-6
depth_raw = np.zeros_like(sharp, dtype=np.float32)
depth_raw[mask] = np.clip((sharp[mask] - vmin) / (vmax - vmin), 0, 1)

# Invert & gamma
if invert_depth:
    depth_raw = 1.0 - depth_raw
depth_norm = np.clip(depth_raw, 0, 1) ** gamma_depth

# =========================
# CREATE BASE SAMPLE GRID
# =========================
yy, xx = np.mgrid[0:H:grid_step, 0:W:grid_step]
yy = yy.ravel()
xx = xx.ravel()
inside = (A[yy, xx] > 0)
yy = yy[inside].astype(np.float32)
xx = xx[inside].astype(np.float32)

# Optionally bias which points we keep based on luminance (brighter => more likely)
if prob_from_brightness > 0:
    p = depth_norm[yy.astype(int), xx.astype(int)]
    # If invert_depth=True, near = bright = small p. We want bias by original brightness (Y), not depth.
    p_bright = (Y[yy.astype(int), xx.astype(int)]).astype(np.float32)  # 0..1
    # Mix constant 1 with brightness to control bias strength
    keep_prob = (1.0 - prob_from_brightness) + prob_from_brightness * p_bright
    rnd = np.random.rand(keep_prob.size)
    keep = rnd < keep_prob
    yy = yy[keep]
    xx = xx[keep]

# =========================
# SLICE QUANTIZATION
# =========================
# Quantize depth into thin planes for a LIDAR look
d = depth_norm[yy.astype(int), xx.astype(int)]
slice_ids = np.minimum((d * depth_slices).astype(np.int32), depth_slices - 1)

# Map slice -> Z (centered, scaled)
cx, cy = W/2.0, H/2.0
slice_z = (np.linspace(0, 1, depth_slices, endpoint=True) - 0.5) * (min(W, H) * depth_scale)
Z = slice_z[slice_ids]

# =========================
# 3D ROTATION & PROJECTION
# =========================
X = xx - cx
Yc = yy - cy

rx = np.deg2rad(rot_x_deg)
ry = np.deg2rad(rot_y_deg)

# rotate X first
Y2 =  np.cos(rx)*Yc - np.sin(rx)*Z
Z2 =  np.sin(rx)*Yc + np.cos(rx)*Z
X2 =  X
# then Y
X3 =  np.cos(ry)*X2 + np.sin(ry)*Z2
Z3 = -np.sin(ry)*X2 + np.cos(ry)*Z2
Y3 =  Y2

# perspective
f = float(focal_len)
den = (f + Z3)
den[den < 1e-3] = 1e-3
px = (X3 * (f / den)) + cx
py = (Y3 * (f / den)) + cy

# =========================
# DUPLICATE/JITTER FOR VOLUME
# =========================
if dup_particles > 1:
    px = np.repeat(px, dup_particles)
    py = np.repeat(py, dup_particles)
    d  = np.repeat(d,  dup_particles)

    jx = (np.random.rand(px.size) - 0.5) * 2 * slice_jitter_px
    jy = (np.random.rand(py.size) - 0.5) * 2 * slice_jitter_px
    px = px + jx
    py = py + jy

# clamp to canvas
ix = np.rint(px).astype(np.int32)
iy = np.rint(py).astype(np.int32)
inside = (ix >= 0) & (ix < W) & (iy >= 0) & (iy < H)
ix, iy, d = ix[inside], iy[inside], d[inside]

# =========================
# STYLE: COLOR, SIZE, ALPHA BY DEPTH
# =========================
# Near (d~0) -> white, bigger, more alpha
# Far  (d~1) -> blue,  smaller, less alpha
cols = (near_rgb[None,:] * (1.0 - d[:,None]) + far_rgb[None,:] * d[:,None]).astype(np.uint8)
sizes = np.rint(min_size + (1.0 - d) * (max_size - min_size)).astype(np.int32)
alphs = np.rint(min_alpha + (1.0 - d) * (max_alpha - min_alpha)).astype(np.uint8)

# =========================
# RASTERIZE TO TRANSPARENT PNG
# =========================
canvas = np.zeros((H, W, 4), dtype=np.uint8)

# Draw small discs; for size==1 we can write pixels directly
small = sizes <= 1
canvas[iy[small], ix[small], :3] = cols[small]
canvas[iy[small], ix[small],  3] = alphs[small]

# For size >= 2, draw circles
big_ix   = ix[~small]
big_iy   = iy[~small]
big_cols = cols[~small]
big_alph = alphs[~small]
big_rad  = sizes[~small]

for x, y, c, a, r in zip(big_ix, big_iy, big_cols, big_alph, big_rad):
    cv2.circle(canvas, (int(x), int(y)), int(r), (*c.tolist(), int(a)), -1)

Image.fromarray(canvas, mode="RGBA").save(OUTPUT)
print("Saved:", OUTPUT)


Saved: subject_pointcloud_lidar.png
